<a href="https://colab.research.google.com/github/namdori61/colab-playground/blob/master/CNN_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import torch
import torch.nn as nn
from torch.optim  import Adam
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from tqdm import tqdm

In [3]:
torch.cuda.is_available()

True

In [0]:
batch_size = 256
learning_rate = 0.002
num_epoch = 10

In [0]:
mnist_train = datasets.MNIST("/content/gdrive/My Drive/tutorials/data", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = datasets.MNIST("/content/gdrive/My Drive/tutorials/data", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

In [0]:
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

In [0]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,5),
            nn.ReLU(),
            nn.Conv2d(16,32,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3,100),
            nn.ReLU(),
            nn.Linear(100,10)
        )
    
    def forward(self,x):
        out = self.layer(x)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)
        return out

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=learning_rate)

In [9]:
loss_arr = []
for i in tqdm(range(num_epoch), desc='training'):
    for image,label in train_loader:
        x = image.to(device)
        y_ = label.to(device)
        
        optimizer.zero_grad()
        output = model(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
        
    print(loss)
    loss_arr.append(loss.cpu().detach().numpy())

training:  10%|█         | 1/10 [00:03<00:27,  3.02s/it]

tensor(0.0569, device='cuda:0', grad_fn=<NllLossBackward>)


training:  20%|██        | 2/10 [00:05<00:23,  2.91s/it]

tensor(0.0318, device='cuda:0', grad_fn=<NllLossBackward>)


training:  30%|███       | 3/10 [00:08<00:20,  2.87s/it]

...

tensor(0.0057, device='cuda:0', grad_fn=<NllLossBackward>)


training:  90%|█████████ | 9/10 [00:25<00:02,  2.80s/it]

tensor(0.0174, device='cuda:0', grad_fn=<NllLossBackward>)


training: 100%|██████████| 10/10 [00:28<00:00,  2.80s/it]

tensor(0.0146, device='cuda:0', grad_fn=<NllLossBackward>)


In [10]:
correct = 0
total = 0

with torch.no_grad():
    for image, label in test_loader:
        x = image.to(device)
        y_ = label.to(device)
        
        output = model(x)
        output_index = torch.argmax(output,1)
        
        total += label.size(0)
        correct += (output_index == y_).sum().float()

    print("Accuracy of Test Data: {}".format(100*correct/total))

Accuracy of Test Data: 99.02845001220703
